In [13]:
# import
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [14]:
# data pre processing
def data_pre_process(raw,is_train=True):
    data = raw.copy()
    data["f3"] = raw["f3"].map({'low': 0, 'mid': 1, 'high': 2})
    label = None
    
    # 暴力Feature 位置
    loc_f = ['f1', 'f2', 'f4', 'f5', 'f6']
    for i in range(len(loc_f)):
        for j in range(i + 1, len(loc_f)):
            data[f'{loc_f[i]}+{loc_f[j]}'] = data[loc_f[i]] + data[loc_f[j]]
            data[f'{loc_f[i]}-{loc_f[j]}'] = data[loc_f[i]] - data[loc_f[j]]
            data[f'{loc_f[i]}*{loc_f[j]}'] = data[loc_f[i]] * data[loc_f[j]]
            data[f'{loc_f[i]}/{loc_f[j]}'] = data[loc_f[i]] / data[loc_f[j]]
            
    # 暴力Feature 通话
    com_f = ['f43', 'f44', 'f45', 'f46']
    for i in range(len(com_f)):
        for j in range(i + 1, len(com_f)):
            data[f'{com_f[i]}+{com_f[j]}'] = data[com_f[i]] + data[com_f[j]]
            data[f'{com_f[i]}-{com_f[j]}'] = data[com_f[i]] - data[com_f[j]]
            data[f'{com_f[i]}*{com_f[j]}'] = data[com_f[i]] * data[com_f[j]]
            data[f'{com_f[i]}/{com_f[j]}'] = data[com_f[i]] / data[com_f[j]]
            
    # 离散化            
    all_f = [f'f{idx}' for idx in range(1, 47) if idx != 3]
    for col in all_f:
        data[f'{col}_log'] = data[col].apply(lambda x: int(np.log(x)) if x > 0 else 0)

    # 特征交叉        
    log_f = [f'f{idx}_log' for idx in range(1, 47) if idx != 3]
    for i in range(len(log_f)):
        for j in range(i + 1, len(log_f)):
            data[f'{log_f[i]}_{log_f[j]}'] = data[log_f[i]]*10000 + data[log_f[j]]
    
    data = data.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    if is_train:
        label = data["label"]
        del data["label"]
    del data["id"]
    
    return data,label

In [15]:
# go process
# label_data no_label_data semi_label_data
label_data = pd.read_csv("./raw/dataTrain.csv")[:50000]
no_label_data = pd.read_csv("./raw/dataA.csv")
semi_label_data = pd.read_csv("./raw/dataNoLabel.csv")
label = label_data["label"]

data_list = [label_data,no_label_data,semi_label_data]
for index,data in enumerate(data_list):
    if index == 0:
        is_train=True
    else:
        is_train=False
    data_list[index],_ = data_pre_process(data,is_train)

In [16]:
# save data to cache
data_list[0].to_csv("./cache/label_data.csv",index=False)
data_list[1].to_csv("./cache/no_label_data.csv",index=False)
data_list[2].to_csv("./cache/semi_label_data.csv",index=False)
label.to_csv("./cache/label.csv",index=False)